In [1]:
# Librerías para manejo de datos
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree
pd.options.mode.chained_assignment = None  # default='warn'
import pickle

In [2]:
# Ubicacion de la base de datos (CAMBIAR SI SE REQUIERE CARGAR OTROS DATOS)
db_location = r"C:\Users\Martin\OneDrive\Documentos\Sexto semestre\BI\Lab1\enfermedades.csv"

In [3]:
# Leer los datos
df_enfer = pd.read_csv(db_location)
# Dimensiones de los datos
df_enfer.shape

(67500, 19)

In [4]:
df_enfer_t = df_enfer

df_enfer_t = df_enfer_t.dropna()
# Dimensiones de los datos
df_enfer_t.shape

(52337, 19)

In [5]:
# Ver los datos
display(df_enfer_t.sample(5)) # Muestra


,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
8763,8763,29.53,No,No,No,10.0,10.0,No,Female,59-55,White,No,No,Very good,7.0,Yes,No,No,No
2417,2417,21.14,No,No,No,5.0,3.0,No,Female,24-18,White,No,No,Very good,7.0,No,No,No,No
34848,34848,25.73,Yes,No,No,0.0,2.0,No,Male,59-55,White,No,Yes,↺Excellent↺,6.0,No,No,No,No
11180,11180,26.45,No,No,No,0.0,0.0,No,Male,64-60,White,No,Yes,Excellent,8.0,No,No,No,No
50151,50151,27.8,No,No,No,2.0,2.0,No,Female,69-65,White,No,Yes,Very good,8.0,No,No,No,No


In [6]:
display(df_enfer_t.head(5)) # Primeras Filas


,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
0,0,26.5,No,No,No,0.0,0.0,No,Male,64-60,White,No,Yes,Very good,8.0,No,No,No,No
1,1,28.75,ýNoÚ,No,No,0.0,2.0,No,Male,59-55,White,No,Yes,Very good,7.0,No,No,No,No
2,2,19.9,Yes,No,No,0.0,1.0,No,Male,39-35,White,No,Yes,Good,7.0,No,No,No,No
3,3,28.06,No,No,No,1.0,1.0,No,Female,69-65,White,Yes,Yes,Good,8.0,No,No,No,£NoÇ
4,4,29.29,Yes,No,No,0.0,1.0,No,Female,74-70,White,No,Yes,Very good,7.0,Yes,No,No,No


In [7]:
# Imprimimos los diferentes tipos de las columnas
df_enfer_t.dtypes

Index                 int64
BMI                  object
Smoking              object
AlcoholDrinking      object
Stroke               object
PhysicalHealth       object
MentalHealth         object
DiffWalking          object
Sex                  object
AgeCategory          object
Race                 object
Diabetic             object
PhysicalActivity     object
GenHealth            object
SleepTime           float64
Asthma               object
KidneyDisease        object
SkinCancer           object
HeartDisease         object
dtype: object

In [8]:
# Ahora definimos la función que nos va a permitir transformar datos booleanos
def transform_boolean (row,label):
    if row[label] == 'No' or  'No' in str(row[label]):
        dato=0
    elif row[label] == 'Yes' or  'Yes' in str(row[label]):
        dato=1
    else:
        dato=np.NaN
    return dato 

In [9]:
df_enfer_t['Smoking']=df_enfer_t.apply (lambda row: transform_boolean(row,'Smoking'), axis=1) 
df_enfer_t['AlcoholDrinking']=df_enfer_t.apply (lambda row: transform_boolean(row,'AlcoholDrinking'), axis=1) 
df_enfer_t['Stroke']=df_enfer_t.apply (lambda row: transform_boolean(row,'Stroke'), axis=1) 
df_enfer_t['DiffWalking']=df_enfer_t.apply (lambda row: transform_boolean(row,'DiffWalking'), axis=1) 
df_enfer_t['PhysicalActivity']=df_enfer_t.apply (lambda row: transform_boolean(row,'PhysicalActivity'), axis=1) 
df_enfer_t['Asthma']=df_enfer_t.apply (lambda row: transform_boolean(row,'Asthma'), axis=1) 
df_enfer_t['KidneyDisease']=df_enfer_t.apply (lambda row: transform_boolean(row,'KidneyDisease'), axis=1) 
df_enfer_t['SkinCancer']=df_enfer_t.apply (lambda row: transform_boolean(row,'SkinCancer'), axis=1) 

In [10]:
display(df_enfer_t.sample(5)) # Muestra

,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
34223,34223,32.08,0.0,0,0.0,0.0,0.0,0,Male,34-30,Other,No,1,Very good,5.0,0,0,0,No
43511,43511,23.49,0.0,0,0.0,0.0,0.0,0,Female,74-70,White,No,1,Very good,8.0,0,0,0,No
43023,43023,25.06,1.0,0,0.0,0.0,1.0,0,Female,59-55,White,No,1,Good,0.0,0,0,0,No
29675,29675,35.9,1.0,0,0.0,2.0,0.0cpa,1,Female,69-65,Black,No,0,Fair,6.0,0,0,0,No
13018,13018,50.18,0.0,0,0.0,30.0,30.0,1,fmfz,59-55,Black,Yes,0,Poor,6.0,0,0,0,Yes


In [11]:
def transform_age (row):
    try:
        edad = int((str(row['AgeCategory']).split('-'))[0])
    except:
        if row['AgeCategory'] == 'or older 80':
            edad = 80
        else: 
            edad = np.NaN
    return edad

In [12]:
df_enfer_t['AgeCategory']=df_enfer_t.apply (lambda row: transform_age(row), axis=1)  

In [13]:
df_enfer_t['HeartDisease']=df_enfer_t.apply (lambda row: transform_boolean(row,'HeartDisease'), axis=1) 

In [14]:
display(df_enfer_t.sample(5)) # Muestra

,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
24244,24244,23.04,0.0,0,0.0,0.0,0.0,0,Male,69,White,No,1,Good,8.0,0,0,0,0
24994,24994,23.71,0.0,0,0.0,0.0,0.0,0,Male,64,White,No,1,Excellent,6.0,0,0,0,0
39114,39114,27.46,1.0,0,0.0,0.0,10.0,0,Female,29,White,No,1,Very good,7.0,1,0,0,0
33878,33878,46.67,0.0,0,0.0,0.0,0.0,0,Female,39,White,No,0,Good,8.0,0,0,0,0
57061,57061,31.28,0.0,0,0.0,0.0,0.0,0,mc,74,White,No,1,Excellent,7.0,0,0,1,0


In [15]:
def transform_float (row,label, limit):
    
    try:
        flo =float(str(row[label]))
        if flo > limit or flo<0:
            flo = np.NaN
    except:
        flo = np.NaN
    
    return flo

In [16]:
df_enfer_t['BMI']=df_enfer_t.apply (lambda row: transform_float(row,'BMI',99), axis=1)  
df_enfer_t['PhysicalHealth']=df_enfer_t.apply (lambda row: transform_float(row,'PhysicalHealth',30), axis=1) 
df_enfer_t['MentalHealth']=df_enfer_t.apply (lambda row: transform_float(row,'MentalHealth',30), axis=1) 
df_enfer_t['SleepTime']=df_enfer_t.apply (lambda row: transform_float(row,'SleepTime',24), axis=1) 

In [17]:
display(df_enfer_t.sample(5)) # Muestra


,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
5795,5795,33.00,0.0,0,1.0,30.0,0.0,1,Male,79,White,"No, borderline diabetes",0,Fair,5.0,0,1,1,1
11198,11198,20.94,0.0,0,0.0,0.0,0.0,0,Female,69,White,No,1,Very good,7.0,0,0,0,0
37884,37884,30.90,1.0,0,0.0,0.0,30.0,0,Female,39,White,Yes,1,Good,9.0,0,0,0,0
66166,66166,30.13,0.0,0,0.0,0.0,0.0,0,Male,80,White,No,1,Very good,8.0,0,0,0,0
47980,47980,26.63,0.0,0,0.0,21.0,22.0,1,Female,69,White,Yes,0,xs,7.0,0,0,0,1


In [18]:
def transform_string (row, label):
    flo=np.NaN
    if label == 'Sex':
        if 'female' in str(row[label]).lower():
            flo=1
        elif 'male' in str(row[label]).lower():
            flo = 0

    elif label == 'Race':
        if 'white' in str(row[label]).lower():
            flo=1
        elif 'hispanic' in str(row[label]).lower():
            flo = 2
        elif 'black' in str(row[label]).lower():
            flo = 3
        elif 'asian' in str(row[label]).lower():
            flo = 4
        elif 'other' in str(row[label]).lower():
            flo = 5
    elif label == 'Diabetic':
        
        if 'no' in str(row[label]).lower():
            flo = 0
        elif 'yes' in str(row[label]).lower():
            flo = 1
    elif label == 'GenHealth':
        if 'poor' in str(row[label]).lower():
            flo=1
        elif 'fair' in str(row[label]).lower():
            flo = 2
        elif 'very' in str(row[label]).lower():
            flo = 4
        elif 'good' in str(row[label]).lower():
            flo = 3
        elif 'excellent' in str(row[label]).lower():
            flo = 5
    else:
        flo=np.NaN
    return flo 

In [19]:
df_enfer_t['Sex']=df_enfer_t.apply (lambda row: transform_string(row,'Sex'), axis=1) 
df_enfer_t['Race']=df_enfer_t.apply (lambda row: transform_string(row,'Race'), axis=1) 
df_enfer_t['Diabetic']=df_enfer_t.apply (lambda row: transform_string(row,'Diabetic'), axis=1) 
df_enfer_t['GenHealth']=df_enfer_t.apply (lambda row: transform_string(row,'GenHealth'), axis=1) 

In [20]:
display(df_enfer_t.sample(5)) # Muestra

,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
36922,36922,33.00,0.0,0,0.0,0.0,0.0,0,0.0,59,1.0,0.0,0,3.0,7.0,0,0,0,0
56710,56710,45.19,1.0,0,0.0,2.0,5.0,1,1.0,54,1.0,0.0,1,3.0,6.0,0,0,0,0
14386,14386,25.77,1.0,0,0.0,0.0,0.0,0,0.0,54,1.0,0.0,1,3.0,8.0,0,0,0,0
30724,30724,32.27,1.0,0,0.0,0.0,NaN,0,0.0,54,NaN,0.0,1,4.0,8.0,0,0,0,0
39034,39034,NaN,0.0,0,0.0,0.0,0.0,1,0.0,79,1.0,0.0,1,3.0,8.0,0,0,1,0


In [21]:
# Borrar datos nulos
df_enfer_t = df_enfer_t.dropna()
# Dimensiones de los datos
df_enfer_t.shape

(40607, 19)

In [22]:
display(df_enfer_t.sample(5)) # Muestra

,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
50494,50494,25.82,0.0,0,0.0,0.0,0.0,0,1.0,34,1.0,0.0,1,5.0,6.0,0,0,0,0
12939,12939,23.13,0.0,0,0.0,0.0,0.0,0,1.0,59,1.0,0.0,1,4.0,7.0,0,0,0,0
45796,45796,19.35,0.0,0,0.0,0.0,0.0,0,0.0,80,1.0,0.0,0,2.0,8.0,0,0,0,1
13114,13114,27.12,1.0,0,0.0,0.0,0.0,0,1.0,59,1.0,0.0,1,5.0,7.0,0,0,0,0
28970,28970,29.29,0.0,0,0.0,0.0,0.0,0,1.0,49,2.0,0.0,0,4.0,7.0,1,0,0,0


In [23]:
#SE GUARDAN LOS ARCHIVOS MODIFICADOS
#df_enfer_t.to_csv(r'C:\Users\Martin\OneDrive\Documentos\Sexto semestre\BI\Lab1\enfermedades_depurado.csv', header=False, index=False)

In [24]:
display(df_enfer_t.head(5)) # Primeras Filas

,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
0,0,26.50,0.0,0,0.0,0.0,0.0,0,0.0,64,1.0,0.0,1,4.0,8.0,0,0,0,0
1,1,28.75,0.0,0,0.0,0.0,2.0,0,0.0,59,1.0,0.0,1,4.0,7.0,0,0,0,0
2,2,19.90,1.0,0,0.0,0.0,1.0,0,0.0,39,1.0,0.0,1,3.0,7.0,0,0,0,0
3,3,28.06,0.0,0,0.0,1.0,1.0,0,1.0,69,1.0,1.0,1,3.0,8.0,0,0,0,0
4,4,29.29,1.0,0,0.0,0.0,1.0,0,1.0,74,1.0,0.0,1,4.0,7.0,1,0,0,0


In [25]:
#KNN____________________________________________

# Se selecciona la variable objetivo, en este caso "HeartDisease".

# Del conjunto de datos se elimina la variable "HeartDisease e index"

df_enfer_p = df_enfer_t.drop(['Index'], axis=1)
df_enfer_p = df_enfer_p.drop(['Race'], axis=1)
df_enfer_p = df_enfer_p.drop(['DiffWalking'], axis=1)

Y = df_enfer_p['HeartDisease']
X =df_enfer_p.drop(['HeartDisease'], axis=1)

In [26]:
# Dividir los datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [27]:
display(X.head(5)) # Primeras Filas

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,Sex,AgeCategory,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,26.50,0.0,0,0.0,0.0,0.0,0.0,64,0.0,1,4.0,8.0,0,0,0
1,28.75,0.0,0,0.0,0.0,2.0,0.0,59,0.0,1,4.0,7.0,0,0,0
2,19.90,1.0,0,0.0,0.0,1.0,0.0,39,0.0,1,3.0,7.0,0,0,0
3,28.06,0.0,0,0.0,1.0,1.0,1.0,69,1.0,1,3.0,8.0,0,0,0
4,29.29,1.0,0,0.0,0.0,1.0,1.0,74,0.0,1,4.0,7.0,1,0,0


In [28]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh = neigh.fit(X_train, Y_train)
y_pred = neigh.predict(X_test)

In [29]:
# Se genera la matriz de confusión
confusion_matrix(Y_test, y_pred)

array([[7235,  229],
       [ 565,   93]], dtype=int64)

In [30]:
# Mostrar reporte de clasificación
print(classification_report(Y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7464
           1       0.29      0.14      0.19       658

    accuracy                           0.90      8122
   macro avg       0.61      0.56      0.57      8122
weighted avg       0.88      0.90      0.89      8122



In [31]:
print('Exactitud: %.2f' % accuracy_score(Y_test, y_pred))
print("Recall: {}".format(recall_score(Y_test,y_pred)))
print("Precisión: {}".format(precision_score(Y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(Y_test,y_pred)))

Exactitud: 0.90
Recall: 0.1413373860182371
Precisión: 0.2888198757763975
Puntuación F1: 0.18979591836734694


In [32]:
#NORMALIZAMOS LOS DATOS PARA MEJORAR LOS RESULTADOS
df_enfe_pru = df_enfer_p.copy()
df_enfe_pru
normalized_df=(df_enfe_pru-df_enfe_pru.min())/(df_enfe_pru.max()-df_enfe_pru.min())

print(normalized_df)

            BMI  Smoking  AlcoholDrinking  Stroke  PhysicalHealth  \
0      0.191689      0.0              0.0     0.0        0.000000   
1      0.221850      0.0              0.0     0.0        0.000000   
2      0.103217      1.0              0.0     0.0        0.000000   
3      0.212601      0.0              0.0     0.0        0.033333   
4      0.229088      1.0              0.0     0.0        0.000000   
...         ...      ...              ...     ...             ...   
67491  0.327212      1.0              0.0     0.0        0.000000   
67493  0.227614      1.0              0.0     0.0        1.000000   
67494  0.245576      1.0              0.0     0.0        0.000000   
67497  0.204290      0.0              0.0     0.0        0.000000   
67499  0.170777      0.0              0.0     0.0        0.000000   

       MentalHealth  Sex  AgeCategory  Diabetic  PhysicalActivity  GenHealth  \
0          0.000000  0.0     0.714286       0.0               1.0       0.75   
1          

In [33]:
# Se selecciona la variable objetivo, en este caso "HeartDisease".
Y = normalized_df['HeartDisease']
# Del conjunto de datos se elimina la variable "HeartDisease"
X = normalized_df.drop(['HeartDisease'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
neigh = KNeighborsClassifier(n_neighbors=3,p=2) 
neigh = neigh.fit(X_train, Y_train) 
y_pred = neigh.predict(X_test) 
print('Exactitud: %.2f' % accuracy_score(Y_test, y_pred))
print("Recall: {}".format(recall_score(Y_test,y_pred)))
print("Precisión: {}".format(precision_score(Y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(Y_test,y_pred)))

Exactitud: 0.90
Recall: 0.1534954407294833
Precisión: 0.27671232876712326
Puntuación F1: 0.19745845552297167


In [34]:
#Estadisticas con el conjunto de entrenamiento
y_pred = neigh.predict(X_train)
print('Exactitud: %.2f' % accuracy_score(Y_train, y_pred))
print("Recall: {}".format(recall_score(Y_train,y_pred)))
print("Precisión: {}".format(precision_score(Y_train,y_pred)))
print("Puntuación F1: {}".format(f1_score(Y_train,y_pred)))

Exactitud: 0.93
Recall: 0.3907355021216407
Precisión: 0.7203389830508474
Puntuación F1: 0.5066483264557543


In [35]:

# Lista de Hiperparámetros a afinar
n_neighbors = list(range(1,10))
n_odd_neighbors = list(filter(lambda x: (x % 2 != 0), n_neighbors))
p=[1,2] #Función de distancia 1: manhattan, 2: euclidean, otro valor: minkowski

#Convert to dictionary
hyperparameters = dict(n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier()

#Use GridSearch
mejor_modelo_knn = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
mejor_modelo_knn.fit(X_train, Y_train)

#Print The value of best Hyperparameters
print('Best p:', mejor_modelo_knn.best_estimator_.get_params()['p'])
print('Best n_neighbors:', mejor_modelo_knn.best_estimator_.get_params()['n_neighbors'])

Best p: 1
Best n_neighbors: 8


In [36]:
# Obtener el mejor modelo.
neigh_final = mejor_modelo_knn.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred_train = neigh_final.predict(X_train)
y_pred_test = neigh_final.predict(X_test)
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(Y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % accuracy_score(Y_test, y_pred_test))

Exactitud sobre entrenamiento: 0.92
Exactitud sobre test: 0.92


In [37]:
#Guardamos elmodelo en un archivo PKL
pickle.dump(neigh, open(r"C:\Users\Martin\OneDrive\Documentos\Sexto semestre\BI\Lab1\modelo_KNN.pkl", "wb"))
